In [1]:
import cv2
import numpy as np
def blur_faces_video(video_path, output_path, output_extension="mp4"):
    try:
        # Load the deep learning-based face detection model
        prototxt_path = "deploy.prototxt"
        weights_path = "res10_300x300_ssd_iter_140000.caffemodel"
        net = cv2.dnn.readNet(prototxt_path, weights_path)

        # Open the video file
        video_capture = cv2.VideoCapture(video_path)

        if not video_capture.isOpened():
            print(f"Error: Could not open video file '{video_path}'.")
            return False

        # Get the frames per second (fps) of the video
        fps = video_capture.get(cv2.CAP_PROP_FPS)

        # Get the width and height of the video frames
        frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

        while True:
            ret, frame = video_capture.read()

            if not ret:
                break

            # Convert the frame to blob and pass through the network
            blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
            net.setInput(blob)
            detections = net.forward()

            # Iterate through the detections
            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]

                if confidence > 0.5:  # Consider detections with confidence greater than 0.5
                    box = detections[0, 0, i, 3:7] * np.array([frame_width, frame_height, frame_width, frame_height])
                    (startX, startY, endX, endY) = box.astype("int")

                    # Extract the face ROI
                    face = frame[startY:endY, startX:endX]

                    # Apply Gaussian blur to the face
                    blurred_face = cv2.GaussianBlur(face, (99, 99), 0)

                    # Replace the face region with the blurred face
                    frame[startY:endY, startX:endX] = blurred_face

            # Write the modified frame to the output video
            out.write(frame)

        # Release video capture and writer objects
        video_capture.release()
        out.release()

        print(f"Faces blurred successfully and video saved to '{output_path}'.")
        return True

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

# Example usage
video_path = "people_video_1.mp4"  # Replace with your video file path
output_path = "blurred_faces_video.mp4"  # You can change the extension here if needed

if blur_faces_video(video_path, output_path):
    print("Face blurring completed. You can view the modified video using a video player.")


Faces blurred successfully and video saved to 'blurred_faces_video.mp4'.
Face blurring completed. You can view the modified video using a video player.
